In [47]:
from collections import defaultdict
import re
import json

In [2]:
bool(re.sub(' ','', '  '))

False

In [21]:
recs = []
rec_dict = {}
with open('wos/savedrecs.txt') as f:
    # er delimits records
    for line in f:
        if re.match('ER', line):
            recs.append(rec_dict)
            rec_dict = {}
        else:
            if line[0:2] == '  ':
                if type(rec_dict[current_key]) != list:
                    rec_dict[current_key] = [rec_dict[current_key]]
                rec_dict[current_key].append(line[3:].rstrip('\n'))
            else:
                rec_dict[line[0:2]] = line[3:].rstrip('\n')
                current_key = line[0:2]

In [22]:
recs

[{'AF': 'Lin, Martin',
  'AU': 'Lin, M',
  'BP': '447',
  'C1': '[Lin, Martin] Rutgers State Univ, New Brunswick, NJ 08901 USA.',
  'CR': ['ADAMS RM, 1979, J PHILOS, V76, P5, DOI 10.2307/2025812',
   'Lamarra Antionio, 1986, INFINTO LEIBNIZ PROB, P173',
   'Broad C. D., 1994, GW LEIBNIZ CRITICAL, V3, P7',
   'Broad C. D., 1975, LEIBNIZ INTRO, P41',
   'BURDICK H, 1991, SYNTHESE, V88, P1, DOI 10.1007/BF00540090',
   'CHERNOFF F, 1981, PHILOS QUART, V31, P126, DOI 10.2307/2218719',
   'Cover Jan, 1999, SUBSTANCE INDIVIDUAT',
   "D'Agostino Fred, 1976, PHILOS Q, V26, P101",
   'Disalle Robert, 2004, CAMBRIDGE COMPANION, P37',
   'Earman John, 1989, WORLD ENOUGH SPACE T, P119',
   'HACKING I, 1979, J PHILOS, V76, P285, DOI 10.2307/2025471',
   'Ishiguro Hide, 1976, LEIBNIZ COLLECTION C, P200',
   'Jauernig Anja, 2008, OXFORD STUDIES EARLY, P191',
   'Jauernig A, 2010, J HIST PHILOS, V48, P171',
   'Jolley Nicholas, 2005, LEIBNIZ, P85',
   'Kulstad Mark, 1980, SO J PHILOS, V18, P424',
   'M

In [17]:
help(object.__hash__)

Help on wrapper_descriptor:

__hash__(self, /)
    Return hash(self).



In [7]:
class Node:
    def __init__(self, name):
        if type(name)!= str:
            print(name, type(name))
            raise Exception('name must be str!')
        self.name = name
    
    def __eq__(self, n2):
        if type(n2)==type(self):
            return self.name == n2.name
        return False
    
    def __hash__(self):
        return hash(self.name)
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return str(self)

class Edge:
    def __init__(self, citer, citee, year, article_id, journal, language, keywords=[]):
        if type(citer) != Node or type(citee) != Node:
            raise Exception('Citer and citee must be Nodes')
        self.citer = citer
        self.citee = citee
        self.year = year
        self.article_id = article_id
        self.journal = journal
        self.language = language
        self.keywords = keywords
    
    def __str__(self):
        return str(self.citer) + ' -> ' + str(self.citee) + ' {} {}'.format(self.year, self.journal)
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, e2):
        if type(e2)==type(self):
            test = self.citer == e2.citer
            test &= self.citee == e2.citee
            test &= self.article_id == e2.article_id
            return test
        return False
    
    def __hash__(self):
        return hash(str(self) + '__' + self.article_id)
        
class Graph:
    def __init__(self, nodes=set(), edges=set()):
        self.nodes = nodes
        self.edges = edges

    def save_edge_list(self, save_attrs=False):
        filename = 'stn_edge_list.csv'
        if save_attrs:
            filename = 'stn_edge_list_attrs.csv'
        with open(filename, 'w') as outfile:
            fieldnames = ['citer', 'citee']
            if save_attrs:
                fieldnames += ['year', 'journal', 'article_id', 'language', 'keywords']
            writer = csv.DictWriter(outfile, fieldnames)
            writer.writeheader()
            for edge in self.edges:
                output_dict = {f: str(getattr(edge, f)) for f in fieldnames}
                writer.writerow(output_dict)
    
    def save_node_list(self, save_attrs=False):
        filename = 'stn_node_list.csv'
        if save_attrs:
            filename = 'stn_node_list_attrs.csv'
        with open(filename, 'w') as outfile:
            fieldnames = ['name']
#             if save_attrs:
            writer = csv.DictWriter(outfile, fieldnames)
            writer.writeheader()
            for node in self.nodes:
                output_dict = {f: getattr(node, f) for f in fieldnames}
                writer.writerow(output_dict)

In [29]:
import re
import csv
        
class wos_processor:
    # create list of nodes and edges
    # if we end up deciding that two nodes are the same, 
    # we can just 
    # have date, journal, institution and full citation attributes on edges
    # wos author index: default format lastname firstinitalsecondinitial
    # http://wcs.webofknowledge.com/SA/getSearchTerm.do?movetoValue=wisan&IMoveTo.x=0&IMoveTo.y=0&termValue=wisan&product=WOS&SID=N1lngK2B9FPb3VwL5Oz&pageNum=0&startDate=&endDate=&timeSpan=All+Years&termName=AU&indexType=browse
    def __init__(self, files, file_dir='wos/philpapers_mostpop'):
        # initialize with a list of filenames
        self.files = files
        self.file_dir = file_dir
        # create a list of dicts to store records
        self.records = []
        self.edges = []
        self.nodes = []
    
    def split_records(self):
        recs = []
        rec_dict = {'CR': ''}
        for file in self.files:
            filepath = self.file_dir + '/' + file
            with open(filepath) as f:
                # 'ER' delimits records
                for line in f:
                    if re.match('ER', line):
#                         if file=='savedrecs104.txt':
#                             print(rec_dict)
                        if type(rec_dict['CR']) != list:
                            rec_dict['CR'] = [rec_dict['CR']]
                        recs.append(rec_dict)
                        rec_dict = {'CR': ''}
                    elif line[0:2] == '  ':
                        if type(rec_dict[current_key]) != list:
                            rec_dict[current_key] = [rec_dict[current_key]]
                        rec_dict[current_key].append(line[3:].rstrip('\n'))
                    else:
                        rec_dict[line[0:2]] = line[3:].rstrip('\n')
                        current_key = line[0:2]
        self.records = recs
    
    def check_keywords(self):
        kw_plus_journals = set()
        kw_journals = set()
        for rec_dict in self.records:
            if 'ID' in rec_dict.keys():
                if type(rec_dict['SO']) == list:
                    print(rec_dict['SO'])
                else:
                    kw_plus_journals.add(rec_dict['J9'])
            if 'DE' in rec_dict.keys():
                if type(rec_dict['SO']) == list:
                    print(rec_dict['SO'])
                else:
                    kw_journals.add(rec_dict['J9'])
        return kw_plus_journals, kw_journals
    
    @staticmethod
    def wos_author_name(name):
        name_parts = re.sub('\.', ',', re.sub('\s','',name)).split(',')
        if len(name_parts) != 2:
            return name_parts[0].lower()
        initials = []
        for part in name_parts[1:]:
            init_list = [char for char in part]
            initials += init_list
        
        initials.insert(0, name_parts[0])

        return ' '.join(initials).lower()
        
    @staticmethod
    def is_article_with_cr(rec_dict):
        if 'DT' not in rec_dict.keys() or 'CR' not in rec_dict.keys():
#                 a few nonsense entries with no dt - ignore
#                 print(rec_dict)
                return False
        if rec_dict['DT'].lower()[:7] != 'article':
            # stuff i downloaded without specifying article only - ignore
#                 print(rec_dict['DT'])
                return False
        return True
    
    @staticmethod
    def get_keywords_list(rec_dict):
        author_keywords = rec_dict['DE'] if 'DE' in rec_dict.keys() else []
        keywords_plus = rec_dict['ID'] if 'ID' in rec_dict.keys() else []
        if author_keywords:
            if type(author_keywords)==list:
                author_keywords = [kw for kw in [line.split(';') for line in author_keywords]]
            else:
                author_keywords = author_keywords.split(';')
            
        if keywords_plus:
            if type(keywords_plus)==list:
                keywords_plus = [kw for kw in [line.split(';') for line in keywords_plus]]
            else:
                keywords_plus = keywords_plus.split(';')
        return author_keywords + keywords_plus

    def create_edges(self):
        for rec_dict in self.records:
            if not self.is_article_with_cr(rec_dict):
                continue
            if type(rec_dict['AU']) == list:
                citers = [self.wos_author_name(name) for name in rec_dict['AU']]
#                 print(citer, rec_dict['AU'])
            else:
                citer = rec_dict['AU']
                citers = [self.wos_author_name(citer)]
                if citer == '[Anonymous]':
                    # ignore anonymous articles
                    continue
            # format last_name, first_initsecond_init
            #   '  ' secondauthorlast_name, first_initsecond_init
            #full name = rec_dict['AF']
            journal = rec_dict['SO']
            
            journal_abbrev = rec_dict['J9']
            year = rec_dict['PY']
            language = rec_dict['LA']
            title = rec_dict['TI']
            abstract = rec_dict['AB'] if 'AB' in rec_dict.keys() else None
            # keywords plus is additional keywords that were not listed by author or publisher
            # added by thomson reuters editors to article entries
            #             # format:
            #             keywords_plus = rec_dict['ID']
            keywords = self.get_keywords_list(rec_dict)
            article_id = rec_dict['UT']
            citers = [Node(citer) for citer in citers]
            for cit in rec_dict['CR']:
                entries = cit.split(',')
                if len(entries) > 1:
                    citee, reference_year = entries[0], entries[1]
                    ref = ','.join(entries[2:])
                name_parts = citee.split(' ')
                if len(name_parts) == 1:
                # just a last name
                    author_ref = Node(name_parts[0].lower())
                    if author_ref == '[anonymous]':
                        continue
                elif len(name_parts) >= 2:
                    name_parts = [name_parts[0]] + [forename[0] for ix, forename in enumerate(name_parts) if ix>0 and forename]
                    #         if name_parts[0].lower() == 'leibniz':
                    #             print(name_parts)
                    author_ref = Node(' '.join(name_parts).lower())
                self.nodes.append(author_ref)
                for citer in citers:
#                     if article_id == 'WOS:A1994QB12400003':
#                         print(author_ref, year, article_id, journal_abbrev, language, keywords, type(keywords))
                    self.nodes.append(citer)
                    self.edges.append(Edge(citer, author_ref, year, article_id, journal_abbrev, language, keywords))
            if article_id == 'WOS:A1979JC64200002':
                print(rec_dict['CR'])
        self.nodes = list(set(self.nodes))
#       this should still allow citations from same author - due to __eq__ on edge including article id
        self.edges = list(set(self.edges))

In [30]:
filenames = ['savedrecs.txt'] + ['savedrecs ({}).txt'.format(i) for i in range(1,68)]
filenames += ['savedrecs{}.txt'.format(i) for i in range(68,116)]
print(filenames)
# filenames = ['savedrecs (41).txt']
ws1 = wos_processor(filenames)
ws1.split_records()
ws1.create_edges()
# kwp, kw = ws1.check_keywords()
# ws1.records

['savedrecs.txt', 'savedrecs (1).txt', 'savedrecs (2).txt', 'savedrecs (3).txt', 'savedrecs (4).txt', 'savedrecs (5).txt', 'savedrecs (6).txt', 'savedrecs (7).txt', 'savedrecs (8).txt', 'savedrecs (9).txt', 'savedrecs (10).txt', 'savedrecs (11).txt', 'savedrecs (12).txt', 'savedrecs (13).txt', 'savedrecs (14).txt', 'savedrecs (15).txt', 'savedrecs (16).txt', 'savedrecs (17).txt', 'savedrecs (18).txt', 'savedrecs (19).txt', 'savedrecs (20).txt', 'savedrecs (21).txt', 'savedrecs (22).txt', 'savedrecs (23).txt', 'savedrecs (24).txt', 'savedrecs (25).txt', 'savedrecs (26).txt', 'savedrecs (27).txt', 'savedrecs (28).txt', 'savedrecs (29).txt', 'savedrecs (30).txt', 'savedrecs (31).txt', 'savedrecs (32).txt', 'savedrecs (33).txt', 'savedrecs (34).txt', 'savedrecs (35).txt', 'savedrecs (36).txt', 'savedrecs (37).txt', 'savedrecs (38).txt', 'savedrecs (39).txt', 'savedrecs (40).txt', 'savedrecs (41).txt', 'savedrecs (42).txt', 'savedrecs (43).txt', 'savedrecs (44).txt', 'savedrecs (45).txt', '

In [41]:
[e for e in  ws1.edges if e.article_id == 'WOS:A1979JC64200002']
len(ws1.edges)
g = Graph(ws1.nodes, ws1.edges)
print(len(g.edges), len(g.nodes))
g.save_node_list()
g.save_edge_list()
g.save_edge_list(True)

854438 101592


In [46]:
with open('stn_edge_list_attrs.csv', 'r') as f:
#     WOS:A1979JC64200002
#     WOS:A1994QB12400003
    reader = csv.DictReader(f)
    count = 0
    for row in reader:
        if row['citer'] == 'heidegger m' or row['citee'] == 'heidegger m':
            print(row['citer'], row['citee'])
count

makkreel r a heidegger m
christoff d heidegger m
darwall s heidegger m
kisiel t heidegger m
nancy j l heidegger m
engelland c heidegger m
waldenfels b heidegger m
dostal r j heidegger m
miller j a heidegger m
schalow f heidegger m
karagiannis n heidegger m
crowell s heidegger m
dreyfus h heidegger m
kleinberg e heidegger m
zahavi d heidegger m
schwemmer o heidegger m
belu d s heidegger m
zahavi d heidegger m
engelland c heidegger m
lahav r heidegger m
dastur f heidegger m
vanbrakel j heidegger m
stoller s heidegger m
lamb a w heidegger m
harries k heidegger m
stone a heidegger m
sun r heidegger m
wright k heidegger m
vandepitte m m heidegger m
caputo j d heidegger m
thorsteinsson b heidegger m
preston b heidegger m
owens w d heidegger m
clifton-soderstrom k heidegger m
lennon k heidegger m
shockey r m heidegger m
edwards p heidegger m
marconi d heidegger m
macavoy l heidegger m
sokolowski r heidegger m
weiss f g heidegger m
crowell s g heidegger m
schmitt r heidegger m
seeburger f f he

0

In [43]:
854438*2

1708876

In [26]:
js = {e.journal for e in ws1.edges}

In [29]:
js

{'AM PHILOS QUART',
 'ANALYSIS',
 'ANALYSIS-UK',
 'AUSTRALAS J PHILOS',
 'BIOETHICS',
 'BRIT J AESTHET',
 'BRIT J PHILOS SCI',
 'CAN J PHILOS',
 'CONT PHILOS REV',
 'DIALECTICA',
 'ERKENNTNIS',
 'ETHICS',
 'EUR J PHILOS',
 'GRAZER PHILOS STUD',
 'GRAZER PHILOS STUDIE',
 'INQUIRY',
 'INT J PHILOS STUD',
 'J HIST PHILOS',
 'J PHILOS',
 'J PHILOS LOGIC',
 'J POLIT PHILOS',
 'LINGUIST PHILOS',
 'MIDWEST STUD PHILOS',
 'MIND',
 'MIND MACH',
 'NOUS',
 'P ARISTOTELIAN SOC',
 'PAC PHILOS QUART',
 'PHILOS BOOK',
 'PHILOS COMPASS',
 'PHILOS IMPRINT',
 'PHILOS ISSUES',
 'PHILOS PERSPECT',
 'PHILOS PHENOMEN RES',
 'PHILOS PSYCHOL',
 'PHILOS QUART',
 'PHILOS REV',
 'PHILOS SCI',
 'PHILOS STUD',
 'PHILOS TOPICS',
 'PHILOSOPHIA',
 'RATIO',
 'SOUTHERN J PHILOS',
 'SYNTHESE'}

In [25]:
h = [e for e in ws1.edges if e.citee.name == 'geuss r']
h

[huddleston a -> geuss r  1999 J HIST PHILOS,
 thomson i -> geuss r  1999 CONT PHILOS REV,
 hales s d -> geuss r  1997 AM PHILOS QUART,
 darby d -> geuss r  1981 CAN J PHILOS,
 owen d -> geuss r  1981 EUR J PHILOS,
 nielsen k -> geuss r  IDEA CRITICAL THEORY RATIO,
 geuss r -> geuss r  2002 EUR J PHILOS,
 varga s -> geuss r  2005 DIALECTICA,
 reginster b -> geuss r  1981 PHILOS PHENOMEN RES,
 jenkins s -> geuss r  INTRO BIRTH TRAGEDY J HIST PHILOS,
 sreenivasan g -> geuss r  1981 PHILOS REV,
 zamosc g -> geuss r  2002 EUR J PHILOS,
 huddleston a -> geuss r  1999 INQUIRY,
 sangiovanni a -> geuss r  2008 J POLIT PHILOS,
 shapiro g -> geuss r  1981 BRIT J AESTHET,
 wiens d -> geuss r  2008 J POLIT PHILOS,
 huddleston a -> geuss r  2008 J HIST PHILOS,
 pettit p -> geuss r  1981 ETHICS,
 hardimon m o -> geuss r  1981 J PHILOS,
 schwartz j -> geuss r  1981 CAN J PHILOS,
 jenkins s -> geuss r  1999 J HIST PHILOS,
 reginster b -> geuss r  1981 EUR J PHILOS,
 campbell e -> geuss r  1981 ETHICS,

In [6]:
# I think what we're going to have to do with the author's names is to
# reduce everything to the form last_name first_initial ?second_initial
# then if we have multiple second initials for a single first initial 
# maybe throw everything out maybe have a think.
# e.g. Bell J. L. vs. Bell J. S. (John Stewart)
# if we only have a single candidate second initial for each first initial
# e.g. Beall JC, Beall J.C., Beall J
# or Beebe J.R., Beebe J., Beebe JR
# then just treat the expanded form with two initials
# and the form with one initial as the same person.
# and then compare these reduced, aggregated last_name first_initial forms
# against dbpedia
len([c for c in citations if len(c.split(',')) > 1])

975006

In [7]:
[d for d in ['a', 'b', 'c'] if d=='a']

['a']

In [8]:
author_counts = defaultdict(int)
for c in citations:
    entries = c.split(',')
    if len(entries) > 1:
        author, year = entries[0], entries[1]
        ref = ','.join(entries[2:])
    name_parts = author.split(' ')
    if len(name_parts) == 1:
        # just a last name
        author_counts[name_parts[0].lower()] += 1
    elif len(name_parts) >= 2:
        name_parts = [name_parts[0]] + [forename[0] for ix, forename in enumerate(name_parts) if ix>0 and forename]
#         if name_parts[0].lower() == 'leibniz':
#             print(name_parts)
        author_ref = ' '.join(name_parts).lower()
        author_counts[author_ref] += 1

In [9]:
higher_deg =sorted([(a,c) for a, c in author_counts.items() if c > 5], key=lambda x: x[1], reverse=True)
print(len(higher_deg))
higher_deg

16773


[('[anonymous]', 12638),
 ('lewis d', 9742),
 ('davidson d', 5484),
 ('putnam h', 4438),
 ('kant i', 4071),
 ('wittgenstein l', 3941),
 ('russell b', 3889),
 ('fodor j', 3739),
 ('kripke s', 3256),
 ('frege g', 3005),
 ('rawls j', 2857),
 ('quine w', 2774),
 ('dummett m', 2718),
 ('kim j', 2683),
 ('dretske f', 2668),
 ('carnap r', 2628),
 ('burge t', 2611),
 ('aristotle', 2490),
 ('hume d', 2442),
 ('goldman a', 2406),
 ('williams b', 2381),
 ('jackson f', 2377),
 ('harman g', 2335),
 ('wright c', 2307),
 ('williamson t', 2274),
 ('mcdowell j', 2268),
 ('heidegger m', 2209),
 ('searle j', 2118),
 ('nagel t', 2107),
 ('stalnaker r', 2062),
 ('goodman n', 1983),
 ('quine w v', 1975),
 ('hintikka j', 1963),
 ('chisholm r', 1951),
 ('shoemaker s', 1880),
 ('quine w v o', 1844),
 ('dennett d', 1814),
 ('chomsky n', 1784),
 ('kitcher p', 1775),
 ('popper k', 1771),
 ('husserl e', 1745),
 ('parfit d', 1712),
 ('kaplan d', 1699),
 ('field h', 1687),
 ('sellars w', 1684),
 ('peacocke c', 1681)

In [256]:
author_counts['ward d']

2